In [ ]:
model = "qwen/qwen3-30b-a3b"
tailscale_server = "https://desktop-3oeimac.tail3b962f.ts.net"
chat_completion_api = tailscale_server + "/api/v0/chat/completions"

In [33]:
transcr_location = "./transcriptions/The_Secrets_of_bodyweight_only_parsed_text.txt"

with open(transcr_location) as file:
    f = file.read()
    print(f)

 How is bodyweight strength training different for adults compared to children? Good question. First seminar I ever did for adults. I'm a national team coach. And what I learned is it really skewed my expectations of what was normal because that was my world. That was who I was surrounded by. They weren't exceptional. They just were. These are the expectations. I ask them to do things and they can perform at this level. So I run an event for adults, first GB seminar on their schedule for the day. And I'm going to take them through just a gentle warmup, nothing hard, nothing demanding. I've got 15 minutes on the schedule, 10 minutes, 15 if we're dragging. I can't get them to focus. They're not paying attention, whatever. And it took an hour and a half for a nothing warmup. They were wrecked from it. They were wrecked. They were exhausted, sweat everywhere, body strewn all over the floor. I remember turning to my staff and I go, you know, what do I do now? You know, they failed warmup. Y

In [42]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=3200, chunk_overlap=40)

docs = text_splitter.split_text(f)

In [43]:
len(docs)

6

In [44]:
print(docs[0])

 How is bodyweight strength training different for adults compared to children? Good question. First seminar I ever did for adults. I'm a national team coach. And what I learned is it really skewed my expectations of what was normal because that was my world. That was who I was surrounded by. They weren't exceptional. They just were. These are the expectations. I ask them to do things and they can perform at this level. So I run an event for adults, first GB seminar on their schedule for the day. And I'm going to take them through just a gentle warmup, nothing hard, nothing demanding. I've got 15 minutes on the schedule, 10 minutes, 15 if we're dragging. I can't get them to focus. They're not paying attention, whatever. And it took an hour and a half for a nothing warmup. They were wrecked from it. They were wrecked. They were exhausted, sweat everywhere, body strewn all over the floor. I remember turning to my staff and I go, you know, what do I do now? You know, they failed warmup. Y

In [26]:
def format_reasoning_model_prompt(prompt:str,no_think=True):
    if no_think:
        return prompt + " /no_think"
    return prompt

In [40]:
import json
with open("./prompt_catalog.json") as catalog_file:
    json_data = json.load(catalog_file)
    
print(json_data.get('system'))

{'transcript_summarizer': 'You are part of a system whose focus is giving practical strategies for increasing performance in workouts relating to bodyweight and gymnastics. \n <ROLE> \n Your primary role is to assist users by summing up transcripts videos given by the user as input, focusing on finding practices which have been useful in workout plans. Summing up means that the generated summary should not be longer of the provided input. Therefore try being more concise, not too verbose and straight to the point. \n </ROLE>'}


In [ ]:
from langchain_openai import ChatOpenAI

# Initialize the LLM pointing to your LM Studio server
llm = ChatOpenAI(
    base_url=f"{tailscale_server}/v1",  # LM Studio server URL
    api_key="lm-studio",  # Can be any string for local models
    model=model,  # Can be any string for local models
)

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(json_data.get('system').get('transcript_summarizer')),
    HumanMessage(format_reasoning_model_prompt(docs[0])),
]

response = llm.invoke(messages)


print(response.content)

<think>

</think>

Adult bodyweight training differs from children's in focus, intensity, and expectations. Adults often lack foundational mobility and strength, requiring more emphasis on warmup, basic positions, and controlled movements. Children typically have better flexibility and natural athleticism, allowing for more advanced drills. For adults, progress is measured by effort and consistency rather than immediate performance, and training must be adapted to their lower baseline capacity. The key is building a strong foundation through physical preparation, mobility, and structured progression.


In [46]:
messages = [
    SystemMessage(json_data.get('system').get('transcript_summarizer')),
    HumanMessage(format_reasoning_model_prompt(docs[0])),
    ]
    
responses = []
for doc in docs:
    messages[1] = HumanMessage(format_reasoning_model_prompt(doc))
    response = llm.invoke(messages)

    print(response.content)
    responses.append(response.content)

<think>

</think>

Adult bodyweight training differs from children's in focus, intensity, and expectations. Adults often lack foundational mobility and strength, requiring more gradual progress. Warmups for adults can be exhausting if they're not prepared, unlike children who are naturally more agile and responsive. For adults, physical preparation is critical—high-load movements like plyometrics and controlled rebounds build the structural strength needed for advanced skills. Effort and consistency matter more than natural talent, as success comes from pushing limits over time. The key is adapting training to individual capacity while maintaining high standards.
<think>

</think>

Summary:  
The speaker emphasizes the importance of proper progression in workouts, especially for bodyweight movements. Overloading too soon—like with kipping pull-ups before strength is built—can lead to injury. Adults often lack the constant movement of youth, making joint prep and mobility crucial. Stead

In [49]:
import re

def remove_think_tags(text):
    """
    Remove <think>...</think> blocks from text using regex.
    Handles multiline content and whitespace.
    """
    # Pattern explanation:
    # <think>     - matches opening tag
    # .*?         - matches any characters (non-greedy)
    # </think>    - matches closing tag
    # re.DOTALL   - makes . match newlines too
    pattern = r'<think>.*?</think>'
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text.strip()

In [51]:
idx = 1
with open("summary_transcript.txt", "w") as output_file:
    for response in responses:
        cleaned_response = remove_think_tags(response)
        output_file.write(f"PART {idx}: \n\n" )
        output_file.write(f"{cleaned_response}\n\n")
        idx += 1

In [20]:
!pip install langchain langchain-openai

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
